In [ ]:
from data2 import DaVaDataset
from PIL import Image
import pickle
from io import BytesIO
import clip

clipped, clippreprocess = clip.load("ViT-B/32", device='cuda')
data = pickle.load(open('./data/images.pkl', 'rb'))

In [ ]:
from utils import get_phi_model
from model import DaVa
import torch

phi_model = get_phi_model()
model = DaVa(phi_model=phi_model)
# model.apply_lora_to_proj(64, 128)

for p in model.parameters():
    p.requires_grad = False

In [ ]:
state_dict1 = torch.load('./weights/final_states26000.pkl')
state_dict2 = torch.load('./weights/final_statues26000.pkl')

model.proj.load_state_dict(state_dict1)
model.phi.load_state_dict(state_dict2)

In [ ]:
from torch.cuda.amp import autocast
from tqdm import tqdm


promptpre = "<|system|>\nYou are a helpful assistant.<|end|>\n<|user|>\n<Image>"
promptpre = model.tokenizer(promptpre, return_tensors="pt", add_special_tokens=False).to('cuda')
with autocast():
    image_embeds = model.proj(image)
promptpost = "What can you infer about the environment or setting of the image? <|end|>\n<|assistant|>\n"
promptpost = model.tokenizer(promptpost, return_tensors="pt", add_special_tokens=False).to('cuda')

bos_embeds = model.phi.model.embed_tokens(torch.tensor([model.tokenizer.bos_token_id])).expand(1, -1, -1)  # bsz x 1 x embed_dim
p_before_embeds = model.phi.model.embed_tokens(promptpre.input_ids).expand(1, -1, -1) # 1 x s1 x embed_dim
p_after_embeds = model.phi.model.embed_tokens(promptpost.input_ids).expand(1, -1, -1)  # 1 x s2 x embed_dim

combined_embeds = torch.cat([bos_embeds, p_before_embeds, image_embeds, p_after_embeds], dim=1).to('cuda')

max_length = 30

predicted_caption = torch.full((1, max_length), 32007)

for pos in tqdm(range(max_length - 1)):

    with autocast():
        model_output_logits = model.phi.forward(inputs_embeds = combined_embeds)['logits'] # 4,49,51200
    predicted_word_token_logits = model_output_logits[:, -1, :].unsqueeze(1) # 4,1,51200
    predicted_word_token = torch.argmax(predicted_word_token_logits, dim = -1) # 4,1
    predicted_caption[:,pos] = predicted_word_token.view(1,-1).to('cpu')
    next_token_embeds = model.phi.model.embed_tokens(predicted_word_token) # 4,1,2560
    combined_embeds   = torch.cat([combined_embeds, next_token_embeds], dim=1)

    if predicted_word_token == 32007:
        break

    del model_output_logits
    del predicted_word_token_logits
    del predicted_word_token
    del next_token_embeds

print(model.tokenizer.decode(predicted_caption[0]))